## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-10-49-11 +0000,bbc,Over-70s face driving ban for failing eye tests,https://www.bbc.com/news/articles/c5yllgezjk3o...
1,2025-08-11-10-48-54 +0000,bbc,Dozens of Malian soldiers arrested over allege...,https://www.bbc.com/news/articles/c79llr41y3go...
2,2025-08-11-10-47-52 +0000,nyt,"Miguel Uribe, Colombian Senator Shot at Campai...",https://www.nytimes.com/2025/08/11/world/ameri...
3,2025-08-11-10-34-44 +0000,bbc,China rams own warship while chasing Philippin...,https://www.bbc.com/news/articles/cev22n0lm1xo...
4,2025-08-11-10-27-12 +0000,nyt,"‘Fighting Fire With Fire,’ Democrats on Defens...",https://www.nytimes.com/2025/08/10/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
91,trump,31
98,putin,12
138,gaza,10
24,china,10
247,ukraine,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
12,2025-08-11-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Trump-Putin summit ...,https://nypost.com/2025/08/11/us-news/ny-postc...,74
121,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-singled-out...,74
83,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,73
99,2025-08-10-23-18-32 +0000,nypost,Intel CEO Lip-Bu Tan to visit White House on M...,https://nypost.com/2025/08/10/business/intel-c...,65
136,2025-08-10-19-52-29 +0000,nypost,Vance says only Trump can decide when to bring...,https://nypost.com/2025/08/10/us-news/vance-sa...,64


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
12,74,2025-08-11-10-00-00 +0000,nypost,NY POSTcast Daily Debrief: Trump-Putin summit ...,https://nypost.com/2025/08/11/us-news/ny-postc...
161,45,2025-08-10-16-41-54 +0000,nyt,Dmitri Kozak Was a Key Putin Aide. He Lost Pow...,https://www.nytimes.com/2025/08/10/world/europ...
121,43,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-singled-out...
24,40,2025-08-11-09-13-10 +0000,wapo,Israeli strike kills Al Jazeera journalist Ana...,https://www.washingtonpost.com/world/2025/08/1...
90,32,2025-08-11-00-09-23 +0000,nypost,"Nvidia, AMD to pay 15% of China chip sale reve...",https://nypost.com/2025/08/10/business/nvidia-...
165,30,2025-08-10-16-15-23 +0000,nypost,Netanyahu lays out Gaza City incursion goals a...,https://nypost.com/2025/08/10/world-news/netan...
4,28,2025-08-11-10-27-12 +0000,nyt,"‘Fighting Fire With Fire,’ Democrats on Defens...",https://www.nytimes.com/2025/08/10/us/politics...
83,28,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...
73,26,2025-08-11-02-31-24 +0000,nypost,Girlfriend of US tourist shot dead during Puer...,https://nypost.com/2025/08/10/us-news/girlfrie...
174,25,2025-08-10-14-32-24 +0000,nypost,UK police arrest nearly 500 at protest for ban...,https://nypost.com/2025/08/10/world-news/uk-po...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
